In [ ]:
# import statements
import deepxde as dde
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import reference_solution as refsol
import tensorflow as tf

# must run code twice for it to actually use this backend if it has to switch
dde.backend.set_default_backend("tensorflow")
dde.config.set_default_float("float32")





In [ ]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']

# Define x values for plotting
x_QLC = np.linspace(-L,L,nx_crystal)



In [ ]:
# define helper functions

def QLC_model(xs, ys):
    """Defines QLC model. Acts as collocation point loss function.

    dNtot_dt = NQLL*surface_diff_coefficient + w_kin*sigma_m \n
    dNQLL_dt = dNtot/dt - (NQLL - NeqQLL)

    Args:
        xs: xs[0] = t, xs[1] = x
        ys: ys[0] = Ntot, ys[1] = NQLL

    Returns:
        [Ntot-loss, NQLL-loss]

    """
    t, x = xs[:, 0:1], xs[:, 1:]
    Ntot, Nqll = ys[:, 0:1], ys[:, 1:]
    
    # Compute gradients
    dNtot_dt = dde.grad.jacobian(ys, xs, i=0, j=0)
    dNqll_dt = dde.grad.jacobian(ys, xs, i=1, j=0)

    # Supersaturation reduction at center
    c_r = GI['c_r']

    # Diffusion coefficient
    D = GI['D']
    D_units = GI['D_units']
    D = AssignQuantity(D,D_units)
    surface_diff_coefficient = D.magnitude

    # Thickness of monolayers
    h_pr = GI['h_pr']
    h_pr_units = GI['h_pr_units']
    h_pr = AssignQuantity(h_pr,h_pr_units)
    h_pr.ito('micrometer')

    # Deposition velocity
    nu_kin = GI['nu_kin']
    nu_kin_units = GI['nu_kin_units']
    nu_kin = AssignQuantity(nu_kin,nu_kin_units)

    # Difference in equilibrium supersaturation between microsurfaces I and II
    sigma0 = GI['sigma0']

    # Supersaturation at facet corner
    sigmaI_corner = GI['sigmaI_corner']

    # Time constant for freezing/thawing
    tau_eq = GI['tau_eq']
    tau_eq_units = GI['tau_eq_units']
    tau_eq = AssignQuantity(tau_eq,tau_eq_units)
    
    # Compute Nqll_eq
    Nbar = GI['Nbar']
    Nstar = GI['Nstar']
    Nqll_eq = (Nbar - Nstar*np.sin(2*np.pi*Ntot))

    # Compute omega_kin
    nu_kin_mlyperus = nu_kin/h_pr
    nu_kin_mlyperus.ito('1/microsecond')
    omega_kin = nu_kin_mlyperus.magnitude * tau_eq.magnitude

    # Compute sigma_m
    sigmaI = sigmaI_corner*(c_r*(x_QLC/L)**2+1-c_r)
    m = (Nqll - (Nbar - Nstar))/(2*Nstar)
    sigma_m = (sigmaI - m * sigma0)  # TODO - unsure

    return [dNtot_dt - ((Nqll*surface_diff_coefficient) + (omega_kin*sigma_m)), 
            dNqll_dt - (dNtot_dt - (Nqll - Nqll_eq))]

In [ ]:
# Save the model as a directory ./model_name/
keras_model = model.net
model_name = "lorenz-t-f64-20x8-200k-pseudo-4000points-decayLR-IC-weighted100000x"
tf.keras.models.save_model(keras_model, model_name, include_optimizer=False, save_format="tf")

# Plot
dde.saveplot(losshistory, train_state, issave=False, isplot=True)